# Pytorch e imágenes de 3D

Tradicionalmente uno puede aplicar diferentes procesos y funciones a imágenes en 2D. En Deep Learning, es
habitual tratar de utilizar **segmentación de objectos**, y **detección de objectos**. En Pytorch existen varios
tipos de herramientas para llevar a cabo estos procesos.

Sin embargo, en los últimos años se han desarrollado varios tipos de modelos que se pueden utilizar para
el análisis y modelaje de objetos en 3D.

Por ejemplo existen las siguientes redes / arquitecturas para objetos en 3D:
- Nube de puntos (Point clouds): [PointNet](http://stanford.edu/~rqi/pointnet), [DGCNN](https://liuziwei7.github.io/projects/DGCNN)
- Redes de registro: [PointNetLK](https://github.com/hmgoforth/PointNetLK), [DCP](https://arxiv.org/abs/1905.03304)

, entre otras más.

Para más información, pueden ver el siguiente artículo: [https://medium.com/@nabil.madali/introduction-to-3d-deep-learning-740c199b100c](https://medium.com/@nabil.madali/introduction-to-3d-deep-learning-740c199b100c)


## Representaciones de imágenes en 3D

Objectos en 3D pueden ser representados en diferentes formas. La siguiente imagen muestra como un conejo
puede ser visualizado en 3D de diferentes maneras:
- a) Point cloud
- b) Voxel Grid
- c) Triangular Grid
- d) multi-view representation

<img src="https://miro.medium.com/max/1400/0*jXcPM_tkNY1_lk55.png" alt="neural_network" style="width: 700px;" align="center"/>

- Nube de puntos (Point clouds):
    - Es una colección de puntos en 3D, en el cual cada punto se puede representar con coordenadas de X, Y, y Z.
    - Similarmente se pueden atribuir más características del punto como color (RGB) y vectores normales (nx, ny, nz).
    - Esta es la forma original de la data para carros autónomos ([LIDAR](https://medium.com/swlh/lidar-the-eyes-of-an-autonomous-vehicle-82c6252d1101))
   
- Voxel grid:
    - Derivado de la nube de puntos, el Voxel grid es como un pixel en 3D.
- Mesh:
    - Superficie que consiste de polígonos convexos con vertices que unen cada uno de los polígonos.
    - Las nubes de puntos pueden derivadas de estas superficies.
- Representación de varios ángulos:
    - Esta forma de representación es una colección de imágenes de 2D del objeto en 3D.

Esto muestra que existen varias formas de representar un objecto en 3D, y existen herramientas y paquetes
para poder visualizarlas y estudiarlas.

## Visualización de datos

Para esta parte, veremos como visualizar unos objetos en 3D del model [ModelNet40](https://modelnet.cs.princeton.edu/)

### Descargando la data
Primero descargaremos la data y la descomprimiremos:


In [ ]:
# Instalando los paquetes necesarios
!python -m pip install -r https://raw.githubusercontent.com/vcalderon2009/2021_06_Deep_Learning_tutorial/master/pkg_requirements.txt -q

# Importando los paquetes necesarios
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px
import open3d as o3d
from pathlib import Path
import plotly.graph_objects as go
import plotly

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

import itertools
import numpy as np
import matplotlib.pyplot as plt

# plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot

Ahora podemos descargar la data:

In [ ]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
!unzip -q ModelNet10.zip

Ahora extraemos la metadata del archivo y construimos un diccionarios con las clases de la data:

In [ ]:
# Definimos el camino de los archivos
path = Path("ModelNet10")

# Folder con la data
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
# Y extraemos las etiquetas de las diferentes imágenes
classes = {folder: ii for ii, folder in enumerate(folders)}

classes

El siguiente paso es crear utilidades para leer los archivos:

In [ ]:
def read_off(file):
    """
    Función para leer archivos `.off`, los cuales tienen los vertices y "caras" de
    cada uno de los objetos.
    """
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    
    return verts, faces

Por ejemplo podemos leer uno de los archivos:

In [ ]:
with open("ModelNet10/desk/train/desk_0107.off", "r") as f:
    verts, faces = read_off(f)

Cada vertice consiste en 3 diferentes arrays que representan los coordenadas de cada punto.
Similarmente cada cara consiste de 3 arrays con indices de cada vértice:

In [ ]:
# Función para visualizar las imágines
# Download data set from plotly repo
def visualize_mesh_plotly(points_arr, faces_arr=None, color="lightpink", opacity=0.5):
    # Extrayendo coordenadas
    x, y, z = points_arr.T
    if faces_arr is not None:
        i, j, k = faces_arr.T
    else:
        i = j = k = None
    # Dibujando plot
    fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, color=color, opacity=opacity)])
    fig.update_layout(
        margin=dict(l=40, r=20, t=40, b=30),
        paper_bgcolor="white",
    )
    fig.show()

In [ ]:
visualize_mesh_plotly(np.asarray(verts), faces_arr=np.asarray(faces))

Pero lo que queremos es convertirlo a una *nube de puntos*. Para ello definimos 2 funciones:

In [ ]:
# Definimos una función que nos deja ver solamente los puntos del objeto
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig


def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

In [ ]:
pcshow(*np.asarray(verts).T)


Esta es la representación de la imagenen como una **nube de puntos**. Una forma de modificar esta data es
distribuyendo uniformente los puntos de esta nube. Por ello utilizamos una funcíon para hacer "sampling" de la data
para crear una nueva nube de puntos:

In [ ]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points

Digamos que queremos **3000 puntos** como nuestra nube de puntos:

In [ ]:
# Aplicamos transformación
pointcloud = PointSampler(3000)((verts, faces))

In [ ]:
print(">>> Número original de puntos: '{0}'".format(np.asarray(verts).shape[0]))
print(">>> Número nuevo de puntos: '{0}'".format(pointcloud.shape[0]))

# Pl

Y podemos visualizar la nueva estructura:

In [ ]:
pcshow(*pointcloud.T)

### Normalización de la data

Uno de los métodos para *preparar* la data es normalizarla con respecto los valores.
En este caso, queremos normalizar la data por el valor máximo y por el promedio de los valores. Esto
es como poner a la imagen dentro de una esfera:

In [ ]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

In [ ]:
norm_pointcloud = Normalize()(pointcloud)

In [ ]:
pcshow(*norm_pointcloud.T)

### Aumentación de data

Similar a 2D, uno puede *aumentar* la data al aplicar transformaciones a la data. En este caso rotaremos las nubes de puntos y
agregaremos **ruido** a las nubes:

In [ ]:
class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud

In [ ]:
rot_pointcloud = RandRotation_z()(norm_pointcloud)
noisy_rot_pointcloud = RandomNoise()(rot_pointcloud)

In [ ]:
pcshow(*noisy_rot_pointcloud.T)

### Otras utilidades

En este caso, definiremos una función para las tranformaciones y para convertir un `numpy.ndarray` a un tensor:

In [ ]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

In [ ]:
def default_transforms():
    return transforms.Compose([
                                PointSampler(1024),
                                Normalize(),
                                ToTensor()
                              ])

Por ejemplo:

In [ ]:
noisy_rot_pointcloud

In [ ]:
ToTensor()(noisy_rot_pointcloud)

### Dataset y Dataloader

Ya que hemos definido las funciones necesarias para aplicar a nuestra data, ahora tenemos
que definir nuestra data para alimentar al modelo. Por ello, vamos a crear nuestra clase
para las nubes de puntos:

In [ ]:
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms()):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform if not valid else default_transforms()
        self.valid = valid
        self.files = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    self.files.append(sample)

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        verts, faces = read_off(file)
        if self.transforms:
            pointcloud = self.transforms((verts, faces))
        return pointcloud

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud = self.__preproc__(f)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}

Definimos nuestra data de entrenamiento:

In [ ]:
train_transforms = transforms.Compose([
                    PointSampler(1024),
                    Normalize(),
                    RandRotation_z(),
                    RandomNoise(),
                    ToTensor()
                    ])

Y separamos la data entre "entrenamiento" y "validación":

In [ ]:
train_ds = PointCloudData(path, transform=train_transforms)
valid_ds = PointCloudData(path, valid=True, folder='test', transform=train_transforms)

Para uso nuestor, creamos un diccionario que contiene los índices y nombres de cada una de las diferentes
clases / etiquetas:

In [ ]:
inv_classes = {i: cat for cat, i in train_ds.classes.items()};
inv_classes

In [ ]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'].size())
print('Class: ', inv_classes[train_ds[0]['category']])

Y por último, definimos nuestro **DataLoader**:

In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

### Modelo

El modelo que utilizaremos se llama **PointNet** y es un modelo que trata de clasificar
nubes de puntos.

<img src="https://miro.medium.com/max/700/1*6ovpVlWKU3ZKk2OT_WKZHA.png" alt="neural_network" style="width: 700px;" align="center"/>


Las siguientes funciones y clases definirán la arquitectura de **PointNet** y será este modelo el cuál usaremos
para entrenar:

In [ ]:
class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix
    
class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64
    
class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

Por último, definiremos nuestra función de pérdida:

In [ ]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

#### Entrenando el modelo

El último paso es entrenar modelo con la data de `ModelNet40`:

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Inicializamos el modelo
pointnet = PointNet()
pointnet.to(device);

Definimos nuestro optimizador:

In [ ]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)

Y escribimos nuestra función para que el modelo se entrene:

In [ ]:
def train(model, train_loader, val_loader=None,  epochs=15, save=True):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
                    print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                        (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                    running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        if save:
            torch.save(pointnet.state_dict(), "save_{0}.pth".format(epoch))

Empezamos con el entrenamiento:

In [ ]:
train(pointnet, train_loader, valid_loader,  save=False)

#### Test / Validación

Por último queremos ver que tan bien se entrenó el modelo con la data:

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
pointnet = PointNet()
pointnet.load_state_dict(torch.load('save.pth'))
pointnet.eval();

In [ ]:
all_preds = []
all_labels = []
with torch.no_grad():
    for i, data in enumerate(valid_loader):
        print('Batch [%4d / %4d]' % (i+1, len(valid_loader)))
                   
        inputs, labels = data['pointcloud'].float(), data['category']
        outputs, __, __ = pointnet(inputs.transpose(1,2))
        _, preds = torch.max(outputs.data, 1)
        all_preds += list(preds.numpy())
        all_labels += list(labels.numpy())

In [ ]:
cm = confusion_matrix(all_labels, all_preds);
cm

Y ahora podemos visualizar los resultados:

In [ ]:
# function from https://deeplizard.com/learn/video/0LhiS6yu2qQ
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Y ahora miremos los resultados:

In [ ]:
plt.figure(figsize=(8,8))
plot_confusion_matrix(cm, list(classes.keys()), normalize=True)

Y sin normalización:

In [ ]:
plt.figure(figsize=(8,8))
plot_confusion_matrix(cm, list(classes.keys()), normalize=False)